In [1]:
# This notebook merges csv data from 
# team box scores and season schedule
#
# Bottom script demonstrates data merged
# successfully for a random sample
#
# NOTE: Script can be reused for other
# merging purposes (e.g., advanced stats, odds, n_days_rest)
#

import pandas as pd
import numpy as np
import os

In [4]:
#
# DONE FOR SEASONS 2000-2001 to CURRENT
#

year_range = range(2019, 1999, -1)
#year_range = range(2013, 2012, -1)

month_range = [10, 11, 12, 1, 2, 3, 4, 5, 6]
#month_range = [11]

day_range = range(1,32)
#day_range = [14]


n_fails = 0

team_file_dir = "data_raw/team_box_scores/"
season_file_dir = "data_raw/season_schedule/"
processed_file_dir = "data_preprocessed/team_box_scores/"

for season_start_year in year_range:
    season_str = str(season_start_year) + "_" \
                 + str(season_start_year + 1)
    if not os.path.exists(processed_file_dir + season_str):
        os.mkdir(processed_file_dir + season_str)

    year = season_start_year
    season_file_path = season_str + "_season_schedule.csv"
    
    # Load season schedule
    sch_pd = pd.read_csv(season_file_dir + season_file_path, \
                         parse_dates=["start_time"])

    
    for month in month_range:
        print("Merging team box score and season data for " \
              + str(year) + " " + str(month))
        for day in day_range:
            if month == 1 and day == 1:
                year = year + 1
#            year = year + 1
            if day < 10:
                day_str = "0" + str(day)
            else:
                day_str = str(day)
            if month < 10:
                month_str = "0" + str(month)
            else:
                month_str = str(month)
                
            team_temp_file_path = "./" + team_file_dir + season_str \
                + "/" + str(year) + "_" + month_str + "_" + day_str \
                + "_" + "team_box_scores.csv"
            processed_temp_file_path = "./" + processed_file_dir \
                + season_str + "/" + str(year) + "_" + month_str \
                + "_" + day_str + "_" + "team_box_scores.csv"
            
            # Checks if team box score data exists for merging
            if os.path.exists(team_temp_file_path):
                
                if not os.path.exists(processed_temp_file_path):

                    # Loads team box scores for specific date
                    team_pd = pd.read_csv(team_temp_file_path)

                    # Finds season schedule data for specific date
                    time_start_str = str(year) + "-" + month_str + "-" \
                                     + day_str + ' 4:00:00'
                    time_start = pd.Timestamp(time_start_str, tz='UTC')
                    time_end = time_start + pd.Timedelta(days=1)
                    idx_start = time_start <= sch_pd["start_time"]
                    idx_end = sch_pd["start_time"] < time_end
                    sch_pd_temp = sch_pd.loc[idx_start & idx_end]

                    
                    # Verifies games match
                    n_teams = team_pd.shape[0]
                    pts_total_1 = 0
                    for i in range(0, n_teams):
                        fg = team_pd.loc[i, "made_field_goals"]
                        three_p = team_pd.loc[i, "made_three_point_field_goals"]
                        ft = team_pd.loc[i, "made_free_throws"]
                        two_p = fg - three_p
                        pts = ft + 2*two_p + 3*three_p
                        pts_total_1 = pts_total_1 + pts
                    pts_total_2 = sch_pd_temp["away_team_score"].sum() \
                                    + sch_pd_temp["home_team_score"].sum()
                    if not pts_total_1 == pts_total_2:
                        print("FAIL at ", time_start, \
                              " Points ", pts_total_1, \
                              "  Points ", pts_total_2)

                    # Merge data by iterating through team and schedule rows
                    n_sch_rows = sch_pd_temp.shape[0]
                    if n_sch_rows > 1:
                        for index, row in sch_pd_temp.iterrows():
                            away_team = row["away_team"]
                            away_team_score = row["away_team_score"]
                            home_team = row["home_team"]
                            home_team_score = row["home_team_score"]

                            for i in range(n_teams):
                                if team_pd.loc[i, "team"] == home_team:
                                    team_pd.loc[i, "location"] = "home"
                                    if home_team_score > away_team_score:
                                        team_pd.loc[i, "outcome"] = "win"
                                    else:
                                        team_pd.loc[i, "outcome"] = "loss"
                                    team_pd.loc[i, "game_score"] = home_team_score
                                    team_pd.loc[i, "opponent"] = away_team
                                    team_pd.loc[i, "opponent_score"] = away_team_score
                                elif team_pd.loc[i, "team"] == away_team:
                                    team_pd.loc[i, "location"] = "away"
                                    if home_team_score < away_team_score:
                                        team_pd.loc[i, "outcome"] = "win"
                                    else:
                                        team_pd.loc[i, "outcome"] = "loss"
                                    team_pd.loc[i, "game_score"] = away_team_score
                                    team_pd.loc[i, "opponent"] = home_team
                                    team_pd.loc[i, "opponent_score"] = home_team_score
                    else:
                        away_team = sch_pd_temp["away_team"].values[0]
                        away_team_score = sch_pd_temp["away_team_score"].values[0]
                        home_team = sch_pd_temp["home_team"].values[0]
                        home_team_score = sch_pd_temp["home_team_score"].values[0]

                        for i in range(n_teams):
                            if team_pd.loc[i, "team"] == home_team:
                                team_pd.loc[i, "location"] = "home"
                                if home_team_score > away_team_score:
                                    team_pd.loc[i, "outcome"] = "win"
                                else:
                                    team_pd.loc[i, "outcome"] = "loss"
                                team_pd.loc[i, "game_score"] = home_team_score
                                team_pd.loc[i, "opponent"] = away_team
                                team_pd.loc[i, "opponent_score"] = away_team_score
                            elif team_pd.loc[i, "team"] == away_team:
                                team_pd.loc[i, "location"] = "away"
                                if home_team_score < away_team_score:
                                    team_pd.loc[i, "outcome"] = "win"
                                else:
                                    team_pd.loc[i, "outcome"] = "loss"
                                team_pd.loc[i, "game_score"] = away_team_score
                                team_pd.loc[i, "opponent"] = home_team
                                team_pd.loc[i, "opponent_score"] = home_team_score

                    # Saves merged data to csv file
                    team_pd.to_csv(processed_temp_file_path, index=False)
                                
                else:
                    print(str(year) + "_" + month_str + "_" \
                          + day_str + ": Game data already merged")



Merging team box score and season data for 2019 10
Merging team box score and season data for 2019 11
Merging team box score and season data for 2019 12
Merging team box score and season data for 2019 1
Merging team box score and season data for 2020 2
Merging team box score and season data for 2020 3
Merging team box score and season data for 2020 4
Merging team box score and season data for 2020 5
Merging team box score and season data for 2020 6
Merging team box score and season data for 2018 10
Merging team box score and season data for 2018 11
Merging team box score and season data for 2018 12
Merging team box score and season data for 2018 1
Merging team box score and season data for 2019 2
Merging team box score and season data for 2019 3
Merging team box score and season data for 2019 4
Merging team box score and season data for 2019 5
Merging team box score and season data for 2019 6
Merging team box score and season data for 2017 10
Merging team box score and season data for 

Merging team box score and season data for 2001 11
Merging team box score and season data for 2001 12
Merging team box score and season data for 2001 1
Merging team box score and season data for 2002 2
Merging team box score and season data for 2002 3
Merging team box score and season data for 2002 4
Merging team box score and season data for 2002 5
Merging team box score and season data for 2002 6
Merging team box score and season data for 2000 10
Merging team box score and season data for 2000 11
Merging team box score and season data for 2000 12
Merging team box score and season data for 2000 1
Merging team box score and season data for 2001 2
Merging team box score and season data for 2001 3
Merging team box score and season data for 2001 4
Merging team box score and season data for 2001 5
Merging team box score and season data for 2001 6


In [6]:
team_file_path = "data_raw/team_box_scores/2018_2019/"
team_file_name = "2018_10_16_team_box_scores.csv"
season_file_path = "data_raw/season_schedule/"
season_file_name = "2018_2019_season_schedule.csv"
processed_file_path = "data_preprocessed/team_box_scores/"
processed_file_name = "2018_2019" + "/" + str(2018) \
                        + "_" + str(10) + "_" + str(16) + "_" \
                        + "team_box_scores.csv"
team_raw = pd.read_csv(team_file_path + team_file_name)
team_preproc = pd.read_csv(processed_file_path + processed_file_name)
season = pd.read_csv(season_file_path + season_file_name, \
                     parse_dates=["start_time"])

In [7]:
team_raw

,team,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls
0,PHILADELPHIA 76ERS,240,34,87,5,26,14,23,6,41,18,8,5,16,20
1,BOSTON CELTICS,240,42,97,11,37,10,14,12,43,21,7,5,14,20
2,OKLAHOMA CITY THUNDER,240,33,91,10,37,24,37,16,29,21,12,6,14,21
3,GOLDEN STATE WARRIORS,240,42,95,7,26,17,18,17,41,28,7,7,21,29


In [8]:
season.head()

,start_time,away_team,away_team_score,home_team,home_team_score
0,2018-10-17 00:00:00+00:00,PHILADELPHIA 76ERS,87,BOSTON CELTICS,105
1,2018-10-17 02:30:00+00:00,OKLAHOMA CITY THUNDER,100,GOLDEN STATE WARRIORS,108
2,2018-10-17 23:00:00+00:00,MILWAUKEE BUCKS,113,CHARLOTTE HORNETS,112
3,2018-10-17 23:00:00+00:00,BROOKLYN NETS,100,DETROIT PISTONS,103
4,2018-10-17 23:00:00+00:00,MEMPHIS GRIZZLIES,83,INDIANA PACERS,111


In [9]:
team_preproc

,team,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,location,outcome,game_score,opponent,opponent_score
0,PHILADELPHIA 76ERS,240,34,87,5,26,14,23,6,41,18,8,5,16,20,away,loss,87.0,BOSTON CELTICS,105.0
1,BOSTON CELTICS,240,42,97,11,37,10,14,12,43,21,7,5,14,20,home,win,105.0,PHILADELPHIA 76ERS,87.0
2,OKLAHOMA CITY THUNDER,240,33,91,10,37,24,37,16,29,21,12,6,14,21,away,loss,100.0,GOLDEN STATE WARRIORS,108.0
3,GOLDEN STATE WARRIORS,240,42,95,7,26,17,18,17,41,28,7,7,21,29,home,win,108.0,OKLAHOMA CITY THUNDER,100.0
